### Ferramental para analise de causa raiz de problemas de rede

In [1]:
from minisom import MiniSom
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16.8,6.72
import math
import sklearn
from sklearn.preprocessing import MinMaxScaler
import sklearn
from scipy.stats import zscore
import somoclu
import seaborn
#import Orange
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import inconsistent
from Orange.projection.pca import PCA
from scipy.cluster.hierarchy import fcluster
from sklearn.utils import shuffle
from sklearn.neighbors.kde import KernelDensity
from time import time
import sompy
from sklearn import preprocessing

C:\Anaconda2\envs\py36\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [29]:
#filedata= 'D:/Development/mestrado/uma/dataset/callsinfo.csv'
filedata= 'D:/Development/causeanalysis/kpis&fails_20170423_20170429_cpe_centro.csv'
df = pd.read_csv(filedata, engine='python', sep=';')
df.head()

,acc_rrc,fails_acc_rrc,eff_cs,fails_acc_cs,acc_cs,eff_ps,fails_acc_ps,acc_ps,eff_hsdpa,fails_acc_hsdpa,...,vs_rab_failestabps_iubfail,vs_rab_failestabps_uufail,vs_rab_failestabps_cong,vs_rrc_rej_redir_dist,vs_rab_failestabps_srbreset,vs_rab_failestabps_cellupd,vs_rab_failestabps_hsupauser_cong,vs_rab_failestabps_hsdpauser_cong,vs_rrc_rej_redir_dist_intrarat,vs_rrc_rej_nodebresunavail
0,100.0,0,100.0,0,100.0,100.0,0,100.0,100.0,0,...,0,0,0,0,0,0,0,0,0,0
1,100.0,0,100.0,0,100.0,100.0,0,100.0,100.0,0,...,0,0,0,0,0,0,0,0,0,0
2,100.0,0,100.0,0,100.0,100.0,0,100.0,100.0,0,...,0,0,0,0,0,0,0,0,0,0
3,100.0,0,100.0,0,100.0,100.0,0,100.0,100.0,0,...,0,0,0,0,0,0,0,0,0,0
4,100.0,0,100.0,0,100.0,100.0,0,100.0,100.0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [31]:
len(df)

23160

In [43]:
len(df)

23160

In [40]:
train_samples = int(len(df)*0.4)
t1_train = df[0:train_samples].apply(zscore)
t1_test = df[train_samples:len(df)].apply(zscore)
X_train = np.array(t1_train)
X_train = X_train.astype('float32')
X_test = np.array(t1_test)
X_test = X_test.astype('float32')

#y_train=np.array(df['FaultCause'][0:train_samples])
#y_train = y_train.astype('float32')

#y_test=np.array(df['FaultCause'][train_samples:len(df)])
#y_test = y_test.astype('float32')

#t1['FaultCause'] = df['FaultCause'].values
#shuffle(t1)
train_df = df[0:train_samples]
test_df = df[train_samples:len(df)]

C:\Anaconda2\envs\py36\lib\site-packages\scipy\stats\stats.py:2247: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [41]:
X_train.shape

(9264, 107)

In [44]:
from matplotlib.mlab import PCA
#construct your numpy array of data
#myData = numpy.array(data) 
results = PCA(X_train) 

#this will return an array of variance percentages for each component
results.fracs

#this will return a 2d array of the data projected into PCA space
#results.Y 

LinAlgError: SVD did not converge

In [45]:
#n_rows, n_columns = 100, 160
n_rows, n_columns = 10,10
#som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False)
som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False,initialization="pca",gridtype='rectangular',neighborhood='bubble')
#%time som.train(dataset,epochs=1000, radius0=10, radiusN=2, radiuscooling='linear', scale0=0.1, scaleN=0.01, scalecooling='linear')
%time som.train(X_train,epochs=20000, radius0=2.84, radiusN=1, radiuscooling='linear', scale0=0.1, scaleN=0.01, scalecooling='linear')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [46]:
np.any(np.isnan(df))

False

In [47]:
np.all(np.isfinite(df))

True

In [51]:
df.loc[(~np.isfinite(df)) & df.notnull()]

ValueError: Cannot index with multidimensional key